In [1]:
word_to_number = {
    'one': 1,
    'two': 2,
    'three': 3,
    'four': 4,
    'five': 5
}

import numpy as np
import random
import torch
torch.set_float32_matmul_precision('medium')

alpha = 0.10

import json
import math
import pandas as pd
from matplotlib import pyplot as plt
import os, sys

# !wget https://files.pythonhosted.org/packages/py3/R/R2CCP/R2CCP-0.0.8-py3-none-any.whl
# !pip install R2CCP-0.0.8-py3-none-any.whl --no-deps
import os
os.makedirs('model_paths', exist_ok=True)

# !pip install configargparse pytorch_lightning torchvision
from R2CCP.main import R2CCP

In [2]:
import json
import pandas as pd
import math
from R2CCP.main import R2CCP
import numpy as np
import os
import random
import psutil
import time


def merge_intervals(sample_intervals):
    if not sample_intervals:
        return (1,5)
    lows = [low for low, high in sample_intervals]
    highs = [high for low, high in sample_intervals]
    return (min(lows), max(highs))

def range_modification(y_qlow, y_qup, range_low,  range_up):
    y_qlow = np.clip(y_qlow, range_low, range_up)
    y_qup = np.clip(y_qup, range_low, range_up)
    return y_qlow, y_qup

def run_experiment(X, y, dimension, dataset):

    X = X.to_numpy().astype(np.float32)
    y = y.to_numpy().astype(np.float32)

    split_index = len(X) // 2 

    X_first_half = X[:split_index]
    X_second_half = X[split_index:]
    y_first_half = y[:split_index]
    y_second_half = y[split_index:]

    X_cal, X_test = X_first_half, X_second_half
    y_cal, y_test = y_first_half, y_second_half
    
    if os.path.exists('model_paths/model_save_destination.pth'):
        os.remove('model_paths/model_save_destination.pth')

    model = R2CCP({'model_path': 'model_paths/model_save_destination.pth', 'max_epochs': 100, 'alpha': alpha})
    model.fit(X_cal, y_cal.flatten())
    intervals = model.get_intervals(X_test)
    intervals = [merge_intervals(sample_intervals) for sample_intervals in intervals]

    df = pd.DataFrame({
        'low':    [iv[0] for iv in intervals],
        'up':     [iv[1] for iv in intervals],
        'y_test': y_test
    })

    df.to_csv(f'R2CCP_{dataset}_{dimension}_halfhalf.csv', index=False)
    
    # m = 13
    # target_idx = np.linspace(3, 15, m)-3

    # adjusted_intervals = [
    # [
    #     (
    #         next((num for num in target_idx if abs(low - num) < 1/6), low),
    #         next((num for num in target_idx if abs(high - num) < 1/6), high)
    #     )
    #     for low, high in sample_intervals
    # ]
    # for sample_intervals in intervals]

    # intervals = adjusted_intervals

    in_interval = [
        (low <= y_true <= high)
        for (low, high), y_true in zip(intervals, y_test)
    ]
    coverage_rate  = np.mean(in_interval)
    average_width = np.mean([high - low for low, high in intervals])

    del model
    torch.cuda.empty_cache()
    time.sleep(1) 

    print(f"Width: {average_width:.4f}, Coverage: {coverage_rate:.4f}")

    return average_width, coverage_rate





In [4]:
# folder_path = f'../model_logits/4omini/'

# dataset = 'Summeval'
# data = {}
# for dimension in ["consistency", "coherence", "fluency", "relevance"]:
#     file_path = os.path.join(folder_path, f"Summeval_{dimension}_logits.csv")
#     df = pd.read_csv(file_path)
#     X = df.iloc[:, :-1]
#     y = df.iloc[:, -1]
#     average_width, coverage_rate = run_experiment(X, y, dimension, dataset)

In [ ]:
folder_path = f'../model_logits/4omini/'

dataset = 'Summeval'
data = {}
# for dimension in ["consistency", "coherence", "fluency", "relevance"]:
dimension = "consistency"
file_path = os.path.join(folder_path, f"Summeval_{dimension}_logits.csv")
df = pd.read_csv(file_path)
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

X = X.to_numpy().astype(np.float32)
y = y.to_numpy().astype(np.float32)

split_index = len(X) // 2 

X_first_half = X[:split_index]
X_second_half = X[split_index:]
y_first_half = y[:split_index]
y_second_half = y[split_index:]

X_cal, X_test = X_first_half, X_second_half
y_cal, y_test = y_first_half, y_second_half

if os.path.exists('model_paths/model_save_destination.pth'):
    os.remove('model_paths/model_save_destination.pth')

model = R2CCP({'model_path': 'model_paths/model_save_destination.pth', 'max_epochs': 100, 'alpha': alpha})
model.fit(X_cal, y_cal.flatten())

# Summeval–Consistency, GPT-4o, Lines 7493 & 7498 in response json
item93_10 = [-12.6856, -9.0606, -5.0606003, -1.0606002,-0.43560016]
item93_11 = [-11.674633, -7.674633, -3.674633, -0.5496331, -0.9246331]

X_test = np.array([item93_10, item93_11])
intervals = model.get_intervals(X_test)
intervals = [merge_intervals(sample_intervals) for sample_intervals in intervals]


In [6]:
intervals

[(4.6123738288879395, 5.0), (4.625612258911133, 5.0)]

In [8]:
def softmax(x):
    e_x = np.exp(x - np.max(x)) 
    return e_x / e_x.sum()


softmax_10 = softmax(item93_10)
softmax_11 = softmax(item93_11)
weights = [1, 2, 3, 4, 5]  


weighted_sum_10 = np.sum(np.array(weights) * softmax_10)
weighted_sum_11 = np.sum(np.array(weights) * softmax_11)

print("Softmax of item93_10:", softmax_10)
print("Softmax of item93_11:", softmax_11)
print("Weighted sum of item93_10:", weighted_sum_10)
print("Weighted sum of item93_11:", weighted_sum_11)

Softmax of item93_10: [3.09666397e-06 1.16201458e-04 6.34438274e-03 3.46391595e-01
 6.47144724e-01]
Softmax of item93_11: [8.50970663e-06 4.64614239e-04 2.53670780e-02 5.77351975e-01
 3.96807823e-01]
Weighted sum of item93_10: 4.640558648067441
Weighted sum of item93_11: 4.370485987193722
